In [33]:
import pandas as pd
import datetime
import numpy as np
import pickle
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

# file with all functions
from utils import *

In [34]:
minedex = pd.read_csv('../../data/recent_mindex.csv')
minedex.shape

(1222, 13)

In [35]:
# Acess Site.csv for lat and lon.
site_locs = pd.read_csv('../../data/minedex/Sites.csv')

minedex['CommodityName'] = minedex['CommodityName'].apply(convert_to_atomic)

### DROPPING WIERD COMMODS FOR NOW.
minedex = minedex[~minedex['CommodityName'].isna()]

# get list of unique mineral combinations
list_combinations = minedex.groupby('SiteCode')['CommodityName'].unique().to_list()

# sort alphabetically
list_combinations = [sorted(commods) for commods in list_combinations]

unique_commods = []
for commods in list_combinations:
    if commods not in unique_commods:
        unique_commods.append(commods)

In [36]:
# WAMEX
wamex = pd.read_csv('../../asher/Merged_Ni_lessthan15km.csv',index_col=0)
# wamex.head()

In [37]:
wamex = wamex.drop(columns=['Unnamed: 0_x', 'Unnamed: 0_y', 'lat', 'long', 
                     'minedex_coords', 'minedex_site',
                    'distance'])

wamex = wamex.rename(columns={
    "wamex_lat": "lat",
    "wamex_lon": "lon"
})

wamex = wamex.drop_duplicates()

In [38]:
wamex_unique_minerals = ['Ni', 'Cu', 'Co', 'Pd', 'Au', 'Pt', 'Rh', 'Fe', 'S', 'Ag']
filtered_unique_commods = filter_by_wamex_minerals(unique_commods, wamex_unique_minerals)

all_unique_commods = []
for cl in filtered_unique_commods:
    all_unique_commods.extend(cl)

wamex_commods = list(set(all_unique_commods))
wamex_commods

['Cu', 'Ni', 'Co', 'Au', 'Ag', 'Pd', 'Fe', 'Rh', 'Pt', 'S']

In [39]:
minedex = minedex[['SiteCode', 'Date', 'CommodityName', 'ppm']]

# sum up duplicate ppm values 
minedex['ppm'] = minedex.groupby(['SiteCode', 'CommodityName'])['ppm'].transform('sum')

# drop duplicate ppm values.
minedex = minedex.drop_duplicates()

# Filter by WAMEX available commods
minedex = minedex[minedex['CommodityName'].isin(wamex_commods)]

minedex_piv = minedex[['SiteCode', 'CommodityName', 'ppm']]

unique_idxs = minedex_piv[['SiteCode', 'CommodityName']].drop_duplicates().index

# don't know why these are here.
minedex_piv[~minedex_piv.index.isin(unique_idxs)]
minedex_piv = minedex_piv[minedex_piv.index.isin(unique_idxs)]

### Prediction Set

In [40]:
# create a pivot table by SiteCode
minedex_piv = minedex_piv.pivot(index='SiteCode', columns='CommodityName', values='ppm').rename_axis(None)

In [41]:
minedex_piv.columns = minedex_piv.columns.rename(None)

In [42]:
site_locs

,SiteCode,lat,lon
0,S0000001,-28.565092,122.480745
1,S0000002,-26.787370,118.574407
2,S0000002,-26.787370,118.574407
3,S0000005,-29.690630,121.012802
4,S0000006,-30.239151,121.222783
...,...,...,...
43119,S0239727,-16.560735,124.199010
43120,S0239728,-21.774382,115.063054
43121,S0239729,-16.635728,124.176460
43122,S0239730,-27.057302,120.957395


In [43]:
minedex_piv

,Ag,Au,Co,Cu,Fe,Ni,Pd,Pt,Rh,S
S0000192,NaN,NaN,NaN,NaN,NaN,59000.0,NaN,NaN,NaN,NaN
S0000539,NaN,NaN,NaN,NaN,NaN,20000.0,NaN,NaN,NaN,NaN
S0001174,NaN,NaN,500.0,53.64,237400.0,15000.0,NaN,NaN,NaN,NaN
S0001178,NaN,NaN,NaN,NaN,NaN,18500.0,NaN,NaN,NaN,NaN
S0001188,NaN,NaN,NaN,NaN,NaN,16500.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
S0236343,NaN,NaN,910.0,NaN,NaN,13500.0,NaN,NaN,NaN,NaN
S0236620,NaN,NaN,2300.0,NaN,NaN,10900.0,NaN,NaN,NaN,NaN
S0236621,NaN,NaN,1100.0,NaN,NaN,4500.0,NaN,NaN,NaN,NaN
S0236838,NaN,NaN,700.0,9000.00,NaN,14000.0,NaN,NaN,NaN,NaN


In [44]:
# stupid af workaround to get merge to work.
minedex_piv = minedex_piv.merge(site_locs, left_index=True, right_on='SiteCode')
minedex_piv.drop_duplicates()

,Ag,Au,Co,Cu,Fe,Ni,Pd,Pt,Rh,S,SiteCode,lat,lon
161,NaN,NaN,NaN,NaN,NaN,59000.0,NaN,NaN,NaN,NaN,S0000192,-32.416982,119.689058
430,NaN,NaN,NaN,NaN,NaN,20000.0,NaN,NaN,NaN,NaN,S0000539,-31.011440,121.680000
918,NaN,NaN,500.0,53.64,237400.0,15000.0,NaN,NaN,NaN,NaN,S0001174,-33.633382,120.374303
923,NaN,NaN,NaN,NaN,NaN,18500.0,NaN,NaN,NaN,NaN,S0001178,-30.393502,121.648875
932,NaN,NaN,NaN,NaN,NaN,16500.0,NaN,NaN,NaN,NaN,S0001188,-30.736222,121.802770
...,...,...,...,...,...,...,...,...,...,...,...,...,...
40080,NaN,NaN,910.0,NaN,NaN,13500.0,NaN,NaN,NaN,NaN,S0236343,-32.785963,118.674386
40331,NaN,NaN,2300.0,NaN,NaN,10900.0,NaN,NaN,NaN,NaN,S0236620,-32.095677,121.644431
40332,NaN,NaN,1100.0,NaN,NaN,4500.0,NaN,NaN,NaN,NaN,S0236621,-32.201621,121.664588
40516,NaN,NaN,700.0,9000.00,NaN,14000.0,NaN,NaN,NaN,NaN,S0236838,-31.591497,123.441046


In [426]:
# for each site, make a string of non-null minerals. 
# template: LinearRegression-Ag-Au-Co-Cu-Pd-Pt-lat-lon

minedex_minerals = minedex_piv.drop(columns=['index', 'lat', 'lon'])

minedex_minerals_str = minedex_minerals.apply(commod_string, axis=1)

In [427]:
minedex_minerals_str['SiteCode'] = minedex_piv['index']
minedex_minerals_str

,Ag,Au,Co,Cu,Fe,Ni,Pd,Pt,Rh,S,commod_str,SiteCode
0,NaN,NaN,NaN,NaN,NaN,59000.0,NaN,NaN,NaN,NaN,Ni,S0000192
1,NaN,NaN,NaN,NaN,NaN,20000.0,NaN,NaN,NaN,NaN,Ni,S0000539
2,NaN,NaN,500.0,53.64,237400.0,15000.0,NaN,NaN,NaN,NaN,Co-Cu-Fe,S0001174
3,NaN,NaN,NaN,NaN,NaN,18500.0,NaN,NaN,NaN,NaN,Ni,S0001178
4,NaN,NaN,NaN,NaN,NaN,16500.0,NaN,NaN,NaN,NaN,Ni,S0001188
...,...,...,...,...,...,...,...,...,...,...,...,...
325,NaN,NaN,2300.0,NaN,NaN,10900.0,NaN,NaN,NaN,NaN,Co,S0236620
326,NaN,NaN,1100.0,NaN,NaN,4500.0,NaN,NaN,NaN,NaN,Co,S0236621
327,NaN,NaN,700.0,9000.00,NaN,14000.0,NaN,NaN,NaN,NaN,Co-Cu,S0236838
327,NaN,NaN,700.0,9000.00,NaN,14000.0,NaN,NaN,NaN,NaN,Co-Cu,S0236838


In [428]:
# Add lat and lon
minedex_min_loc = minedex_minerals_str.merge(site_locs, left_on="SiteCode", right_index=True)

In [429]:
minerals_df = minedex_min_loc.drop(columns=['lat', 'lon', 'commod_str', 'SiteCode'])

orig_ni = minerals_df['Ni']

#Ratio transform
minedex_min_loc.loc[:, minerals_df.columns] = minedex_min_loc.loc[:, minerals_df.columns].apply(
                                                            abundant_ratio, abundant_mineral='Ni', axis=1)
# apply Log transform
minedex_min_loc.loc[:, minerals_df.columns] = minedex_min_loc.loc[:,minerals_df.columns].apply(np.log, axis=1)

In [430]:
# keep ref of sitecode to index
site_codes_ref = minedex_min_loc['SiteCode']

In [432]:
# create a list of dataframes by unique commodity groups.
gb = minedex_min_loc.groupby('commod_str')    
list_of_commod_dfs = [gb.get_group(x) for x in gb.groups]

In [433]:
minedex_min_loc.columns

Index(['Ag', 'Au', 'Co', 'Cu', 'Fe', 'Ni', 'Pd', 'Pt', 'Rh', 'S', 'commod_str',
       'SiteCode', 'lat', 'lon'],
      dtype='object')

In [435]:
list_of_site_pred_dfs = []

for commod_group_df in list_of_commod_dfs:
    site_pred_df = create_site_pred(commod_group_df)
    list_of_site_pred_dfs.append(site_pred_df)

Could not find models for Ag-Au-Cu
Could not find models for Au-Cu-Pd-Pt
Could not find models for Au-Cu-Pd-Pt-Rh
Could not find models for Cu
Could not find models for Cu-Pd-Pt-S
Could not find models for Fe
Could not find models for Ni


In [436]:
all_site_preds_df = pd.concat(list_of_site_pred_dfs, axis=0)

In [437]:
# frik knows why the merge creates duplicates!!!!
all_site_preds_df = all_site_preds_df.merge(minedex[['SiteCode', 'Date']], on='SiteCode').drop_duplicates()

In [438]:
all_site_preds_df

,SiteCode,Co_pred_lr,Co_pred_rf,commods_used,Date
0,S0018904,-1.947308,-0.136785,Ag-Au-Co-Cu-Pd-Pt,09/12/2020
7,S0019102,-1.814471,-0.158555,Ag-Au-Co-Cu-Pd-Pt,09/12/2020
14,S0001425,-1.759355,-1.256325,Co,30/06/2018
16,S0001687,-1.819600,-2.273113,Co,29/06/2004
18,S0002179,-1.823366,-2.635242,Co,29/03/2005
...,...,...,...,...,...
307,S0230776,-2.122509,-2.259829,Co-Fe,24/09/2013
310,S0230931,-1.079864,-1.730262,Co-Fe,29/10/2013
313,S0232040,-2.040919,-1.983738,Co-Fe,30/12/2014
316,S0232041,-2.057151,-1.694422,Co-Fe,30/12/2014


In [439]:
# get original Ni values for inveser transform
all_site_preds_df = all_site_preds_df.merge(orig_ni, left_index=True, right_index=True).drop_duplicates()

In [440]:
all_site_preds_df

,SiteCode,Co_pred_lr,Co_pred_rf,commods_used,Date,Ni
0,S0018904,-1.947308,-0.136785,Ag-Au-Co-Cu-Pd-Pt,09/12/2020,59000.000000
7,S0019102,-1.814471,-0.158555,Ag-Au-Co-Cu-Pd-Pt,09/12/2020,28000.000000
14,S0001425,-1.759355,-1.256325,Co,30/06/2018,33000.000000
16,S0001687,-1.819600,-2.273113,Co,29/06/2004,22100.000000
18,S0002179,-1.823366,-2.635242,Co,29/03/2005,22000.000000
...,...,...,...,...,...,...
307,S0230776,-2.122509,-2.259829,Co-Fe,24/09/2013,36400.000000
310,S0230931,-1.079864,-1.730262,Co-Fe,29/10/2013,2700.000000
313,S0232040,-2.040919,-1.983738,Co-Fe,30/12/2014,6400.000000
316,S0232041,-2.057151,-1.694422,Co-Fe,30/12/2014,103000.000000


### Invert ratio and log

In [441]:
all_site_preds_df['Co_pred_lr'] = np.exp(all_site_preds_df['Co_pred_lr'])  * all_site_preds_df['Ni'] 
all_site_preds_df['Co_pred_rf'] = np.exp(all_site_preds_df['Co_pred_rf'])  * all_site_preds_df['Ni'] 

In [442]:
all_site_preds_df

,SiteCode,Co_pred_lr,Co_pred_rf,commods_used,Date,Ni
0,S0018904,8416.795837,51457.330309,Ag-Au-Co-Cu-Pd-Pt,09/12/2020,59000.000000
7,S0019102,4561.875733,23894.530886,Ag-Au-Co-Cu-Pd-Pt,09/12/2020,28000.000000
14,S0001425,5681.143623,9395.041889,Co,30/06/2018,33000.000000
16,S0001687,3582.201833,2276.103298,Co,29/06/2004,22100.000000
18,S0002179,3552.587197,1577.435258,Co,29/03/2005,22000.000000
...,...,...,...,...,...,...
307,S0230776,4358.203452,3799.009092,Co-Fe,24/09/2013,36400.000000
310,S0230931,917.033062,478.542432,Co-Fe,29/10/2013,2700.000000
313,S0232040,831.419108,880.345954,Co-Fe,30/12/2014,6400.000000
316,S0232041,13165.212803,18921.648010,Co-Fe,30/12/2014,103000.000000


In [447]:
pd.set_option('display.max_rows', 500)

# add cobalt on
true_cobalt = minedex[minedex['CommodityName'] == 'Co'][['SiteCode', 'ppm']]
all_site_preds_df = all_site_preds_df.merge(true_cobalt, on='SiteCode')
all_site_preds_df = all_site_preds_df.rename(columns={'ppm': 'Co_true'})

,SiteCode,Date,commods_used,Co_pred_lr,Co_pred_rf,Co_true
0,S0018904,09/12/2020,Ag-Au-Co-Cu-Pd-Pt,8416.795837,5.145733e+04,410.0
1,S0019102,09/12/2020,Ag-Au-Co-Cu-Pd-Pt,4561.875733,2.389453e+04,360.0
2,S0001425,30/06/2018,Co,5681.143623,9.395042e+03,1700.0
3,S0001687,29/06/2004,Co,3582.201833,2.276103e+03,500.0
4,S0002179,29/03/2005,Co,3552.587197,1.577435e+03,2000.0
5,S0002181,29/03/2005,Co,10208.710701,1.056185e+04,2200.0
6,S0002182,29/03/2005,Co,2283.377822,7.470027e+02,2400.0
7,S0002327,29/04/2008,Co,23370.186045,8.797588e+03,400.0
8,S0002811,22/05/1995,Co,2625.661890,3.753763e+03,700.0
9,S0005015,11/04/2005,Co,5014.149829,2.095580e+04,1100.0


In [448]:
all_site_preds_df = all_site_preds_df[['SiteCode', 'Date', 'commods_used', 'Co_pred_lr', 'Co_pred_rf', 'Co_true']]
all_site_preds_df

,SiteCode,Date,commods_used,Co_pred_lr,Co_pred_rf,Co_true
0,S0018904,09/12/2020,Ag-Au-Co-Cu-Pd-Pt,8416.795837,5.145733e+04,410.0
1,S0019102,09/12/2020,Ag-Au-Co-Cu-Pd-Pt,4561.875733,2.389453e+04,360.0
2,S0001425,30/06/2018,Co,5681.143623,9.395042e+03,1700.0
3,S0001687,29/06/2004,Co,3582.201833,2.276103e+03,500.0
4,S0002179,29/03/2005,Co,3552.587197,1.577435e+03,2000.0
5,S0002181,29/03/2005,Co,10208.710701,1.056185e+04,2200.0
6,S0002182,29/03/2005,Co,2283.377822,7.470027e+02,2400.0
7,S0002327,29/04/2008,Co,23370.186045,8.797588e+03,400.0
8,S0002811,22/05/1995,Co,2625.661890,3.753763e+03,700.0
9,S0005015,11/04/2005,Co,5014.149829,2.095580e+04,1100.0
